<a href="https://colab.research.google.com/github/Naaastenka/Project1/blob/main/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [6]:
#Если не работает загрузка on-line
df=pd.read_csv("/content/Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [7]:
def parse_normative(normative, value):
    if pd.isna(normative):
        return "No normative provided"

    if isinstance(value, str) and 'б/цвета' in value:
        return "Colorless"

    match = re.match(r'([<>]=?)?\s*(\d+(\.\d+)?)', normative)
    if match:
        op, num, _ = match.groups()
        num = float(num)
        if op == '<':
            return value < num
        elif op == '<=':
            return value <= num
        elif op == '>':
            return value > num
        elif op == '>=':
            return value >= num

    match = re.match(r'(\d+(\.\d+)?)\s*-\s*(\d+(\.\d+)?)', normative)
    if match:
        lower, _, upper, _ = match.groups()
        lower = float(lower)
        upper = float(upper)
        return lower <= float(value) <= upper

    return "Invalid normative format"

In [8]:
# Применение нормативного анализа и сравнения
df['Result'] = df.apply(lambda row: parse_normative(row['Норматив'], row['Результат анализа']), axis=1)

In [9]:
# Создание сводного столбца
df['Summary'] = df.apply(lambda row: f"По показателю {row['Показатель']} получен результат {row['Результат анализа']} единицы при нормативе {row['Норматив']}. Т.о. по данному показателю результат {'в норме' if row['Result'] == True else 'не в норме'}.", axis=1)


In [10]:
# Установка столбца «Показатель» в качестве индекса
df.set_index('Показатель', inplace=True)

In [11]:
df[['Единица измерений', 'Результат анализа', 'Норматив', 'Summary']]

,Единица измерений,Результат анализа,Норматив,Summary
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,По показателю pH получен результат 8.4 единицы...
Запах,баллы,1,не более 2-3,По показателю Запах получен результат 1 единиц...
Цветность,градусы,б/цвета,не более 30,По показателю Цветность получен результат б/цв...
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,По показателю Жёсткость получен результат 9.19...
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",По показателю Аммиак и аммоний-ион (по азоту) ...
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",По показателю Нитриты (по NO2) получен результ...
Нитраты (по NO3),мг/дм3,24,не более 45,По показателю Нитраты (по NO3) получен результ...
Фосфаты (P),мг/дм3,0.36,"не более 3,5",По показателю Фосфаты (P) получен результат 0....
Хлориды (Cl),мг/дм3,200,не более 350,По показателю Хлориды (Cl) получен результат 2...


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [ ]:
import random
from collections import Counter

In [ ]:
# Созадём корзину с фруктами
fruit_basket = ['апельсин'] * 5 + ['яблоко'] * 4

In [ ]:
# Функция случаного выбора
def pick_fruits(basket):
    return random.sample(basket, 3)

In [ ]:
# Колличество эксперементов
num_experiments = 100000

In [ ]:
# Проведение эксперементов
results = [pick_fruits(fruit_basket) for _ in range(num_experiments)]

In [ ]:
# Подсчет сколько раз все фрукты были апельсинами
count_all_oranges = sum(1 for result in results if Counter(result)['апельсин'] == 3)

In [ ]:
# Вычисляем вероятность
experimental_probability = count_all_oranges / num_experiments
experimental_probability

0.11915

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [ ]:
import numpy as np

In [ ]:
# Определяем количество экспериментов
num_experiments = 100000

In [ ]:
# Определяем функцию для моделирования процесса проверки
def check_two_details():
    # Создаем список деталей: 7 стандартных и 3 нестандартных
    details = ['стандартная'] * 7 + ['нестандартная'] * 3
    # Перемешиваем детали случайным образом
    np.random.shuffle(details)
    # Проверяем, если первая деталь нестандартная, а вторая стандартная
    if details[0] == 'нестандартная' and details[1] == 'стандартная':
        return True
    return False

In [ ]:
# Выполняем эксперименты
results = sum(check_two_details() for _ in range(num_experiments))

In [ ]:
# Вычисляем экспериментальную вероятность
experimental_probability = results / num_experiments

In [ ]:
# Отображаем результат
experimental_probability

0.23219